# GPS

## Intro

These notes follow along with the youtube series demonstrating how to build your own software gps receiver, found here https://www.youtube.com/watch?v=i7JPjgHa7_A&list=PLmlXFuUXRl5BnKM9PM_tT9uIzlwUxGzLb&index=2

Some supplmentrary resources are:
- https://www.youtube.com/watch?v=s-jFprdDcM4

See also the following books:
- *A Software-Defined GPS and Galileo Receiver: A Single-Frequency Approach*, K. Borro, *et al*
- *Global Positioning System*, Per Enge, *et al.*

************


The plan is to expand the notes we already have on software-defined radio, learn some more relevant python, and maybe even put together a version of the software GPS receiver developed in the series.


************


## The Course Acquistition (C/A) (Navigation) Signal

Called 'Course Acquisition' since for civilian usage and not 'Precision' (*ie* military).

The navigation signal is a 50 bit/s signal containing the encoded satellite position and on-board clock time modulated onto the carrier band (*eg* L1 at about 1.5GHz, &c).

The amplitude of the carrier wave of satellite $i$ at time $t$ will be denoted by $f_i(t)$.

The navigation signal bit at time $t$ for satellite $i$ will be denoted $D_i(t)$.

Phase (the horizontal positon of the carrier wave) modulation is used to carry the navigation signal via **binary phase shift keying**.


## Binary Phase Shift Keying

Here we phase modulate and encode binary data by flipping the amplitude (180 phase shifts) to segements of the signal.

A 0 is no change, and 1 is a amplitude flip, so the binary data *eg* $(0,1)$ may be represented as $(1,-1) f_i(t)$. 

This mapping is known as **polar non-return to zero encoding** and will we denote via hats, *eg* $\hat{D}_i(t)$.

In [15]:
# an example of 4 bps 180-degree bpsk

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

### parameters

carrier_freq = 5        # [Hz]
duration = 1.0          # [seconds]
num_bits = 4            # [bps]
samples_per_bit = 250   # mock digitisation

"""
FIXME

- the number of bits isn't a realy paramter since 4 are passed to plot_bpsk and the widgets.interact
- this re-plots the entire thing with any change to the toggles... this is a bad approach

"""

t = np.linspace(0, duration, num_bits * samples_per_bit, endpoint=False)

### ui

# bit flips

bit_toggles = [widgets.ToggleButton(value=True, description='0') for _ in range(num_bits)]

def update_toggle_description(change):
    change.owner.description = '0' if change.new else '1'

for toggle in bit_toggles:
    toggle.observe(update_toggle_description, names='value')

# annontations (guide line)

show_lines_checkbox = widgets.Checkbox(value=True, description='Show Lines')
show_grid_checkbox = widgets.Checkbox(value=True, description='Show Grid')

### plot signal

def plot_bpsk(b0, b1, b2, b3, show_lines, show_grid):
    
    bits = [b0, b1, b2, b3]
    signal = np.array([])

    for i, bit in enumerate(bits):
        t_bit = np.linspace(i * duration / num_bits, (i + 1) * duration / num_bits, samples_per_bit, endpoint=False)
        phase = 0 if bit else np.pi
        wave = np.sin(2 * np.pi * carrier_freq * t_bit + phase)
        signal = np.concatenate((signal, wave))

    
    plt.plot(t, signal, label=f'{carrier_freq} Hz Carrier')
    
    # plot break for each bit
    if show_lines:
        for i in range(1, num_bits):
            plt.axvline(i * duration / num_bits, color='gray', linestyle='--')
        
    # annotate
    plt.title('BPSK example')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    
    if show_grid:
        plt.grid(True)
    
    plt.show()


widgets.interact(
    plot_bpsk,
    b0=bit_toggles[0],
    b1=bit_toggles[1],
    b2=bit_toggles[2],
    b3=bit_toggles[3],
    show_lines=show_lines_checkbox,
    show_grid=show_grid_checkbox
)

interactive(children=(ToggleButton(value=True, description='0'), ToggleButton(value=True, description='0'), To…

<function __main__.plot_bpsk(b0, b1, b2, b3, show_lines, show_grid)>

## Code Division Multiple Access (CDMA)

This process implements **psuedo-random noise** codes, in order to distinguish between satellites all broadcasting on the same band.

The PRN is unique to each satellite, and repeats at about once a millisecond (PRN datarate = 1.023 bps).

The PRN is mixed with the navigation signal via exclusive or (XOR) and the result is modulated onto the carrier wave via bpsk.

Polar non-return to zero encoding of the XOR is equivalent to multiplication.

Hence, the modulated signal may be represented $\hat{D}_i(t) \hat{PRN}_i(t) f_i(t)$.

#### PRN properties

- strong positive autocorrelation spike at 0 and near 0 elsewhere.
- the cross-correlation of any 2 PRNS is always near 0.

### Summary

So given these properties, when we receive on one band mutliple GPS signals (along with some noise), we can:

- calculate the correlation with a copy of one of these satellites PRN codes
- play with some integrals
- use the fact that the navigation message has a much slower data-rate than the PRN
-  align the known PRN code with the recieved

and eventually arrive at a linear expression from which we can detemine the navigation signal by recording over several periods of the PRN, boosting the SNR. 



## GPS Antennas

- hemispherical reception pattern (for almost all sky coverage)
- if active include amplifier (~28db is common)
- tuned to specific bands (*eg* L1)
  

## SDR set-up

- Centre frequency of L1 band: $f_{l1} = 1575.42 [MHz]$
- Bandwidth: for reasons yet to be understood, $BW = 2(PRN datarate) = 2.046 [MHz]$
- Sampling Rate: Nyquist Theorem...

  ```
  the signal can be determined from its samples if the sampling rate is greater than twice the maximum frequency contained in the signal
  ```

In our case, $f_{max} = f + (BW)/2$, so $f_s ~ 3.2 [GHz]$, buutttt max sampling rate of the rtl-sdr is closer to 3 Mhz... some internal heterodyning and clever aliasing to the rescue.

### Aliasing

and **Bandpass undersampling**...
  
TODO

For reasons (

- alias has effective carrier at 1/770
- no overlapping
- ...
)

I am yet to understand, the settled-upon sample rate is $f_s = 2.046 MHz$.


## I/Q Values

In-phase and Quadrature. Recall:
$I(t) = A(t) \cos( \phi(t) )$ and  $Q(t) = A(t) \sin(\phi(t) )$
are the variable amplitudes to the cos and sin components of a representation of the frequency.

And hence
$\phi(t) = \arctan(Q/I)$.

In [27]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

### parameters

carrier_freq = 5        # [Hz]
duration = 1.0          # [seconds]
samples = 1000          # per second
t = np.linspace(0, duration, samples, endpoint=False)
iq_point = {'i': 1.0, 'q': 0.0}


def update_plots(i, q):
    iq_point['i'] = i
    iq_point['q'] = q

    print(f"(I, Q) = ({i},{q}), phi = {np.arctan(q / i)}")

    i_wave = i * np.cos(2 * np.pi * carrier_freq * t)
    q_wave = q * np.sin(2 * np.pi * carrier_freq * t)
    total = i_wave + q_wave

    fig, axs = plt.subplots(2, 1, figsize=(10, 8))
    plt.subplots_adjust(hspace=0.4)

    ### constellation
    
    axs[0].axhline(0, color='gray', lw=0.5)
    axs[0].axvline(0, color='gray', lw=0.5)
    axs[0].plot(iq_point['i'], iq_point['q'], 'ro')
    axs[0].set_xlim(-2, 2)
    axs[0].set_ylim(-2, 2)
    axs[0].set_aspect('equal')
    axs[0].set_xlabel('I')
    axs[0].set_ylabel('Q')

    ### signal
    
    axs[1].plot(t, i_wave, label='I')
    axs[1].plot(t, q_wave, label='Q')
    axs[1].plot(t, total, label='I + Q')

    axs[1].legend(loc="lower right")
    axs[1].set_xlabel('Time')
    axs[1].set_ylabel('Amplitude')

    axs[1].grid(True)

    plt.show()

### sliders

i_slider = widgets.FloatSlider(value=iq_point['i'], min=-2.0, max=2.0, step=0.01, description='I:')
q_slider = widgets.FloatSlider(value=iq_point['q'], min=-2.0, max=2.0, step=0.01, description='Q:')

widgets.interact(update_plots, 
                 i=i_slider, 
                 q=q_slider)


interactive(children=(FloatSlider(value=1.0, description='I:', max=2.0, min=-2.0, step=0.01), FloatSlider(valu…

<function __main__.update_plots(i, q)>

## More on I/Q sampling

Now, suppose the sampling frequency is $f_1$ and the frequency to be sampled is $f_2$, then we may say $f_2 = f_1 + \Delta f$ where $\Delta f$ some difference between them. The recieved signal (simply approximated) is then 
$$
s(t) = A \cos(2 \pi f_2(t))
     = A \cos(2 \pi (f_1(t) + \Delta f(t))
     = A \cos(2 \pi f_1(t)) + 2 \pi \Delta f(t))
$$
*ie* the signal has the frequency of the sampling frequency plus a time-dependent phase term $phi(t) = 2 \pi \Delta f(t)$, which we recall is equivalent to a rotation in the I-Q plane.

That is the I-Q samples are rotated by the carrier wave's phase. This is better understood by a complex representation of the samples $I + jQ$.

- We will need a bit more info here...

## The Acquisition Stage

- For good coorelation and strong signal strength of the navigation message, the known PRN must be phase matched to the incoming.

### Carrier Frequency shift
  
- Signals from GPS will be effected by Doppler shifts, so we do not know the received frquency band exactly.


### Carrier Wipe Off

## Parameter Space

### Non-coherent integration

### Summary

Must determine:
1. the phase of the PRN code,
2. the frequency shift of the carrier wave

This is done by brute force: trying possible values till the strongest navigation signal is seen.

The presence of asatellite si determined by whether the signal is above a threshold. 




## The Tracking Stage

During reception, after acquisition, it is important to keep track off:
1. the frequency of the carrier wave
2. the phase of the carrier wave
3. the phase of the PRN code
